# Bristlecone routing example

For an introduction to routing see the simpler routing_example notebook. In this example we will see how to perform routing for the Google 72 qubit Bristlecone architecture, with automatic rearrangement of qubits on the architecture to minimise the swaps required. Start off by importing everything we need.

In [1]:
import cirq
from pytket import route_circuit_xmon
from cirq.google import Bristlecone, Foxtail

Let's pick a device (after going through the example you can try changing it to Foxtail), and get the qubits from it.

In [2]:
device = Bristlecone
qubits = list(device.qubits)
print(device)

                                             (0, 5)────(0, 6)
                                             │         │
                                             │         │
                                    (1, 4)───(1, 5)────(1, 6)────(1, 7)
                                    │        │         │         │
                                    │        │         │         │
                           (2, 3)───(2, 4)───(2, 5)────(2, 6)────(2, 7)───(2, 8)
                           │        │        │         │         │        │
                           │        │        │         │         │        │
                  (3, 2)───(3, 3)───(3, 4)───(3, 5)────(3, 6)────(3, 7)───(3, 8)───(3, 9)
                  │        │        │        │         │         │        │        │
                  │        │        │        │         │         │        │        │
         (4, 1)───(4, 2)───(4, 3)───(4, 4)───(4, 5)────(4, 6)────(4, 7)───(4, 8)───(4, 9)───(4, 10)
         │        │      

Let's design a circuit, to see routing in action here we make circuit consisting of four lots of quantum fourier transforms. Each qft sub-circuit acts on qubits spread across the architecture (as we might expect if this were being run as a subroutine of a larger algorithm). 

In [3]:
def Qft_4_qubit(q1: cirq.GridQubit, q2: cirq.GridQubit, q3: cirq.GridQubit, q4: cirq.GridQubit) -> cirq.Circuit:
  circuit = cirq.Circuit.from_ops(
      cirq.H(q1),
      cirq.CZ(q1,q2)**0.5,
      cirq.CZ(q1,q3)**0.25,
      cirq.CZ(q1,q4)**0.125,
      cirq.H(q2),
      cirq.CZ(q2,q3)**0.5,
      cirq.CZ(q2,q4)**0.25,
      cirq.H(q3),
      cirq.CZ(q3,q4)**0.125,
      cirq.H(q4)
      )
  return circuit

circuit = Qft_4_qubit(qubits[0],qubits[6],qubits[11],qubits[13])
circuit += Qft_4_qubit(qubits[1],qubits[5],qubits[4],qubits[9])
circuit += Qft_4_qubit(qubits[2],qubits[7],qubits[12],qubits[14])
circuit += Qft_4_qubit(qubits[3],qubits[8],qubits[10],qubits[15])
circuit

(0, 5): ─────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────@─────────────────@────────────@─────────H─────────────────────────────────────────────────────────────────────
                                                                                                                                                                 │                 │            │
(0, 6): ──────────────────────────────────────────────────────────────────────H───@───────@────────@─────────────────────────────────────────────────────────────┼─────────────────┼────────────┼───────────────────────────────────────────────────────────────────────────────
                                                                                  │       │        │                                                             │                 │            │
(2, 8): ──────────────────────────────────────────────────────────────────────────┼───────┼────────┼────────────────────────────────────────────────────@────────┼─────────@───────┼────────H───@^0.125─────────────────────────────────────────────────────────────────────────
                                                                                  │       │        │                                                    │        │         │       │
(4, 1): ─────────────────────────@─────────────────@────────────@─────────H───────┼───────┼────────┼────────────────────────────────────────────────────┼────────┼─────────┼───────┼────────────────────────────────────────────────────────────────────────────────────────────
                                 │                 │            │                 │       │        │                                                    │        │         │       │
(4, 2): ─────────────────────────┼─────────────────┼────────────┼─────────────────┼───────┼────────┼────────────────────────────────────────H───@───────@^0.25───@^0.125───┼───────┼────────────────────────────────────────────────────────────────────────────────────────────
                                 │                 │            │                 │       │        │                                            │                          │       │
(4, 3): ─────────────────────────┼─────────────────┼────────────┼─────────────────┼───────┼────────┼────────────────────────────────────────────@^0.5────────────H─────────@^0.5───@^0.25───────────────────────────────────────────────────────────────────────────────────────
                                 │                 │            │                 │       │        │
(5, 4): ─────────────────────────┼─────────────────┼────────────┼─────────────────┼───────@^0.25───┼─────────@───────H────────@─────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────
                                 │                 │            │                 │                │         │                │
(5, 6): ────────────────@────────┼─────────@───────┼────────H───@^0.125───────────┼────────────────┼─────────┼────────────────┼─────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────
                        │        │         │       │                              │                │         │                │
(5, 7): ────────────────┼────────┼─────────┼───────┼──────────────────────────────┼────────────────@^0.125───┼───────@────────@^0.125───H───────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────
                        │        │         │       │                              │                          │       │
(5, 9): ────────────────┼────────┼─────────┼───────┼──────────────────────────────┼──────────────────────────┼───────┼──────────────────

The circuit consists of single qubit Hadamard gates and parametrized Controlled-Z (or Controlled-Phase) operations. On the real architecture the controlled two-qubit operations can only happen between neighbouring qubits. We need to move the quantum information around using routing.

First we will just add swaps, optimally, without remapping the qubits. Swaps are extra operations, so are costly, especially on near term machines with low decoherence times, so we need to do as few as possible. We can check each two qubit operation in the output circuit to make sure they occur between neighbouring qubits. 

In [4]:
routed_circuit = route_circuit_xmon(circuit, device)
routed_circuit

(0, 5): ────────────────────────────────────────×───────×─────────────────────────────────────────────×───────────×─────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────
                                                │       │                                             │           │
(0, 6): ────────────────H───────────────────────×───────┼─────────────────────────────────────────────×───────────┼─────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────
                                                        │                                                         │
(1, 4): ────────────────────────────────────────────────┼───────×───────×─────────────────────────────────────────┼─────────────────────────────────────────────×───────×───────────────────────────────────────────────────────────────────────────────────────
                                                        │       │       │                                         │                                             │       │
(1, 5): ────────────────────────────────────────────────×───────×───────┼─────────────────────────────────────────×─────────────────────────@───────────────────×───────┼───────────────────────────────────────────────────────────────────────────────────────
                                                                        │                                                                   │                           │
(2, 3): ────────────────────────────────────────────────────────×───────┼───×────────@────────────────────×─────────────────────────────────┼───────────────────×───────┼─────────────────@─────────────────────────────────────────────────────────────────────
                                                                │       │   │        │                    │                                 │                   │       │                 │
(2, 4): ────────────────────────────────────────────────────────┼───────×───×────────┼───────@────────×───┼───────×─────────────────────────┼───────────────────┼───────×─────────────────@^0.25───@─────────H──────────────────────────────────────────────────
                                                                │                    │       │        │   │       │                         │                   │                                  │
(2, 5): ────────────────────────────────────────────────────────┼───×────────────────┼───────@^0.25───×───┼───────┼─────────────────────────@^0.125─────────────┼──────────────────────────────────┼────────────────────────────────────────────────────────────
                                                                │   │                │                    │       │                                             │                                  │
(2, 6): ────────────────────────────────────────────────×───────┼───×────────────────┼────────────────────┼───────┼─────────────────────────────────────────────┼──────────────────────────────────┼────────────────────────────────────────────────────────────
                                                        │       │                    │                    │       │                                             │                                  │
(2, 7): ────────────────────────────────────────×───────×───────┼────────────────────┼────────────────────┼───────┼─────────────────────────────────────────────┼──────────────────────────────────┼────────────────────────────────────────────────────────────
                                                │               │                    │                    │       │                                             │                                  │
(2, 8): ────────────────────────────────────────×───────────────┼────────────────────┼────────────────────┼───────┼────────────────────────

Now we can also use placement, where the qubits the circuit is using are remapped to reduce the number of swaps required. To do this, we can set the `place` flag to `True` in `route_xmon_circuit`, this will remap the qubits if possible.

In [5]:
placed_routed_circuit = route_circuit_xmon(circuit, device, place=True)
placed_routed_circuit

(4, 1): ────────────────────────────────────────────────────────────────────────────×───────×────────────────────────────────────────────────────────────────────────────────×─────────────@────────×───────────────────────────────────────@─────────H───
                                                                                    │       │                                                                                │             │        │                                       │
(4, 2): ────────────────────────────────────────────────────────────────────────────┼───────×───×────────────────────────────────────────────────────────────────────────────┼───×─────────@^0.25───×───────────────────────────────────────┼─────────────
                                                                                    │           │                                                                            │   │                                                          │
(4, 3): ────────────────────────────────────────────────────────────────────────────┼───────────×──────────────────────────────────────────────────────────@─────────×───────┼───×──────────────────────────────────────────────────────────┼─────────────
                                                                                    │                                                                      │         │       │                                                              │
(4, 4): ────────────────────────────────────────────────────────────────────────────┼───────────×──────────────────────────────────────────────────────────@^0.125───×───────┼──────────────────────────────────────────────────────────────┼─────────────
                                                                                    │           │                                                                            │                                                              │
(4, 5): ────────────────────────────────────────────────────────────────────────────┼───────×───×────────────────────────────────────────────────────────────────────────────┼──────────────────────────────────────────────────────────────┼─────────────
                                                                                    │       │                                                                                │                                                              │
(4, 6): ────────────────────────────────────────────────────────────────────────────┼───×───×───×───×────────────────────────────────────────────────────────────────────────┼──────────────────────────────────────────────────────────────┼─────────────
                                                                                    │   │       │   │                                                                        │                                                              │
(4, 7): ────────────────────────────────────────────────────────────────────────────┼───×───×───×───┼────────────────────────────────────────────────────────────────────────┼──────────────────────────────────────────────────────────────┼─────────────
                                                                                    │       │       │                                                                        │                                                              │
(4, 8): ───────────────────────────────────────────────────────────────────×────────┼───────×───────┼────────────────────────────────────────────────────────────────────────┼──────────────────────────────────────────────────────────────┼─────────────
                                                                           │        │               │                                                                        │                                                              │
(4, 9): ───────────────────────────────────────────────────────────────────×────────┼───

How useful was placement? We can check by checking how many moments are in each circuit. Fewer moments means a smaller circuit depth.

In [6]:
print("Moments without placement: ", len(routed_circuit))
print("Moments with placement: ", len(placed_routed_circuit))

Moments without placement:  33
Moments with placement:  27
